# Bit / Phase 重复码与位相联合纠错

## 本 Notebook 的目标

通过这个 Notebook，我们希望直观理解：

1. **位重复码（Bit Repetition Code）**  
   - 如何通过“多数表决”纠正比特翻转错误（bit-flip error）。

2. **“相位重复码”的经典模拟（Phase Repetition Code）**  
   - 虽然真正的相位翻转是量子概念，这里用 0/1 标签做一个经典抽象，帮助理解后续量子相位纠错。

3. **位 + 相位联合纠错结构（BitPhaseJointCode）**  
   - 构造一个类似 Shor 3×3 结构的经典版本：  
     外层纠正“相位标签”，内层纠正“位翻转”，分层完成纠错。
   - 通过中间结果，看到“先纠 bit 再纠 phase”的两级纠错思路。

In [2]:
import sys
from pathlib import Path

def add_project_root(marker_dirs=("classical", "bit_phase_repetition"), max_depth=5):
    """
    从当前工作路径开始，向上最多 max_depth 层，
    找到同时包含 marker_dirs 中所有目录的路径，
    视为项目根目录，并加入 sys.path。
    """
    cur = Path.cwd()
    for _ in range(max_depth):
        if all((cur / d).exists() for d in marker_dirs):
            if str(cur) not in sys.path:
                sys.path.append(str(cur))
            print("已找到项目根目录:", cur)
            return cur
        cur = cur.parent
    raise RuntimeError(
        f"未在向上 {max_depth} 层目录中找到同时包含 {marker_dirs} 的项目根目录，请检查目录结构。"
    )

PROJECT_ROOT = add_project_root()

from bit_phase_repetition.bit_repetition import BitRepetitionCode
from bit_phase_repetition.phase_repetition import PhaseRepetitionCode
from bit_phase_repetition.bit_phase_joint import BitPhaseJointCode

print("模块导入成功：", BitRepetitionCode, PhaseRepetitionCode, BitPhaseJointCode)



已找到项目根目录: c:\iCloud\iCloudDrive\苏州\Codes\qec_sim
模块导入成功： <class 'bit_phase_repetition.bit_repetition.BitRepetitionCode'> <class 'bit_phase_repetition.phase_repetition.PhaseRepetitionCode'> <class 'bit_phase_repetition.bit_phase_joint.BitPhaseJointCode'>


## 1. 位重复码（Bit Repetition Code）直观理解

设想我们要传输一个比特 `m ∈ {0, 1}`，  
传输过程中可能会发生 **比特翻转错误（0↔1）**。

一个最简单的纠错思路是：

> 不只发 1 个比特，而是发 `n` 个一模一样的比特，  
> 接收端对这 `n` 个比特做 **多数表决（majority vote）**。

例如 `n = 3`：
- 发送 `0` 时，编码为 `000`
- 发送 `1` 时，编码为 `111`

接收端看到：
- `000` → 多数是 0 → 判为 0（无错误）
- `001` → 多数是 0 → 判为 0（纠正了 1 个错误）
- `011` → 多数是 1 → 判为 1（纠正了 1 个错误）
- `111` → 多数是 1 → 判为 1（无错误）

**结论：**
- 只要 **最多只有 1 个比特翻转**，3 重重复码就能纠正。
- 一般地，长度为奇数 `n` 的重复码可以纠正 **最多 `(n-1)/2` 个 bit-flip 错误**。

下面我们用代码枚举 `n=3` 的所有情况，看看多数表决的效果。


In [ ]:
from itertools import product

code_3 = BitRepetitionCode(n=3)
print("使用的编码：", code_3)

def majority_decode(block):
    decoded_bit, error_flag = code_3.decode_block(block)
    return decoded_bit, error_flag

all_blocks = list(product([0, 1], repeat=3))

print("所有长度为 3 的 0/1 模式及其多数表决结果：\n")
print("block  -> decoded, error_detected")
for block in all_blocks:
    decoded_bit, error_flag = majority_decode(list(block))
    print(f"{block} -> {decoded_bit}, {error_flag}")


使用的编码： BitRepetitionCode(n=3)
所有长度为 3 的 0/1 模式及其多数表决结果：

block  -> decoded, error_detected
(0, 0, 0) -> 0, False
(0, 0, 1) -> 0, True
(0, 1, 0) -> 0, True
(0, 1, 1) -> 1, True
(1, 0, 0) -> 0, True
(1, 0, 1) -> 1, True
(1, 1, 0) -> 1, True
(1, 1, 1) -> 1, False


## 2. 多比特序列的位重复码：编码、注入错误、解码

在实际应用中，我们往往传输的是一个比特序列，例如：

$$
[m_0, m_1, \dots, m_{k-1}]
$$

对于每个逻辑比特，我们用重复码编码，得到一个更长的“物理比特序列”。

例如 `n = 3`，逻辑比特 `[1, 0, 1, 1]`：

- `1 -> 111`
- `0 -> 000`
- `1 -> 111`
- `1 -> 111`

拼在一起得到：

`111 000 111 111`  共 12 个物理比特。

下面我们通过代码演示：

1. 编码一串逻辑比特；
2. 手动翻转部分物理比特；
3. 用多数表决恢复原始逻辑比特。


In [16]:
logical_bits = [1, 0, 1, 1]
print("原始逻辑比特：", logical_bits)

bit_code = BitRepetitionCode(n=3)

# 1) 编码
encoded_bits = bit_code.encode(logical_bits)
print("编码后物理比特：", encoded_bits)

# 2) 注入错误：翻转若干物理比特（这里只是示例位置）
received_bits = encoded_bits.copy()
error_positions = [1, 4, 9]  # 手动指定几个错误位置
for pos in error_positions:
    received_bits[pos] ^= 1  # 0 -> 1 或 1 -> 0

print("\n注入错误位置：", error_positions)
print("带错误的物理比特：", received_bits)

# 3) 解码
decoded_bits, error_flags = bit_code.decode(received_bits)
print("\n解码结果：", decoded_bits)
print("每个 block 是否检测到错误：", error_flags)
print("是否成功还原原始逻辑比特：", decoded_bits == logical_bits)


原始逻辑比特： [1, 0, 1, 1]
编码后物理比特： [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]

注入错误位置： [1, 4, 9]
带错误的物理比特： [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]

解码结果： [1, 0, 1, 1]
每个 block 是否检测到错误： [True, True, False, True]
是否成功还原原始逻辑比特： True


## 3. “相位重复码”的经典模拟（Phase Repetition Code）

在量子纠错中，除了 **比特翻转错误（X error）** 之外，还有  
**相位翻转错误（Z error）**。

为了在经典世界里“提前练习”这件事，我们做一个抽象：

- 用 `0` 和 `1` 来表示两种“相位标签”，例如：  
  - `0` → “+” 相位  
  - `1` → “-” 相位

在数学上，我们对这些标签做的事情**和位重复码一模一样**：  
还是重复若干次、再多数表决。

区别只是：
- 位重复码更像是在保护“0/1 本身”；
- 相位重复码更像是在保护“+/- 标签”（未来会对接到量子里的 `|+>, |->` 态）。

下面我们用 `PhaseRepetitionCode` 做一个简单实验。


In [17]:
phase_code = PhaseRepetitionCode(n=3)
print("相位重复码：", phase_code)

# 可以理解为四个“相位比特” [+,-,-,+]
phase_bits = [0, 1, 1, 0]  
print("原始相位标签（0/1）：", phase_bits)

# 1) 编码
encoded_phase = phase_code.encode(phase_bits)
print("编码后物理比特：", encoded_phase)

# 2) 注入“相位错误”（仍然用 0/1 翻转来模拟）
received_phase = encoded_phase.copy()
phase_error_positions = [0, 3, 7]
for pos in phase_error_positions:
    received_phase[pos] ^= 1

print("\n注入相位错误位置：", phase_error_positions)
print("带错误的物理比特：", received_phase)

# 3) 解码
decoded_phase, phase_error_flags = phase_code.decode(received_phase)
print("\n解码得到的相位标签：", decoded_phase)
print("每个 block 是否检测到错误：", phase_error_flags)
print("是否成功还原原始相位标签：", decoded_phase == phase_bits)



相位重复码： PhaseRepetitionCode(n=3)
原始相位标签（0/1）： [0, 1, 1, 0]
编码后物理比特： [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]

注入相位错误位置： [0, 3, 7]
带错误的物理比特： [1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0]

解码得到的相位标签： [0, 1, 1, 0]
每个 block 是否检测到错误： [True, True, True, False]
是否成功还原原始相位标签： True


## 4. 位 + 相位联合纠错：BitPhaseJointCode（类 Shor 3×3 结构）

在量子纠错中，有一个非常经典的 **Shor 9 比特码**：

- 用 3 个物理比特纠正位翻转；
- 用 3 组这样的比特块（共 9 比特）再纠正相位翻转；
- 总体上同时防御 X 和 Z 两类错误。

这里我们用一个 **经典版本**来帮助理解这种“分层纠错”的思想：

- 外层：相位重复码，重复 `n_phase` 次  
- 内层：位重复码，对每个“相位比特”再重复 `n_bit` 次  

对单个逻辑比特 `m`，编码后得到 `n_phase * n_bit` 个物理比特：

1. 先构造相位层：`[m, m, ..., m]`（长度 `n_phase`）
2. 对其中每一个再做位重复（长度 `n_bit`），拼成总长度为 `n_phase * n_bit` 的 block

例如 `n_phase = 3, n_bit = 3` 时，一个逻辑比特对应 9 个物理比特。

解码顺序：
1. **先纠 bit-flip**：  
   对每个长度为 `n_bit` 的小 block 用位重复码解码，得到 `n_phase` 个“相位比特”。
2. **再纠 phase**：  
   对这 `n_phase` 个“相位比特”用相位重复码解码，得到最终的逻辑比特。

我们现在用 `BitPhaseJointCode` 展示这个两级纠错的过程。


In [19]:
joint_code = BitPhaseJointCode(n_bit=3, n_phase=3)
print("BitPhaseJointCode:", joint_code)

logical_bits_joint = [1, 0]   # 两个逻辑比特
print("原始逻辑比特：", logical_bits_joint)

# 1) 编码
encoded_joint = joint_code.encode(logical_bits_joint)
print("编码后物理比特长度：", len(encoded_joint))
print("编码后物理比特：", encoded_joint)

# 2) 注入 bit + “phase” 混合错误（本质上都是对 0/1 的翻转）
received_joint = encoded_joint.copy()

# 手工指定一些错误位置（可以随便改）
error_positions_joint = [1, 4, 7, 10, 15]
for pos in error_positions_joint:
    received_joint[pos] ^= 1

print("\n注入错误位置：", error_positions_joint)
print("带错误的物理比特：", received_joint)

# 3) 解码（会同时给出中间的“相位层”信息）
decoded_joint, infos = joint_code.decode(received_joint)

print("\n解码得到的逻辑比特：", decoded_joint)
print("是否成功还原原始逻辑比特：", decoded_joint == logical_bits_joint)

print("\n每个逻辑比特的详细纠错信息：")
from pprint import pprint
for i, info in enumerate(infos):
    print(f"\n逻辑比特 {i}:")
    pprint(info)


BitPhaseJointCode: BitPhaseJointCode(n_bit=3, n_phase=3)
原始逻辑比特： [1, 0]
编码后物理比特长度： 18
编码后物理比特： [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

注入错误位置： [1, 4, 7, 10, 15]
带错误的物理比特： [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0]

解码得到的逻辑比特： [[1], [0]]
是否成功还原原始逻辑比特： False

每个逻辑比特的详细纠错信息：

逻辑比特 0:
{'inner_bit_error_flags': [True, True, True],
 'intermediate_phase_bits': [1, 1, 1],
 'phase_error_detected': False}

逻辑比特 1:
{'inner_bit_error_flags': [True, False, True],
 'intermediate_phase_bits': [0, 0, 0],
 'phase_error_detected': False}


## 5. 随机错误实验：多次试验感受纠错能力

为了更直观地感受编码的“冗余”带来的纠错能力，我们可以：

1. 随机生成一串逻辑比特；
2. 用 BitPhaseJointCode 编码；
3. 随机选择一定数量的位置注入错误；
4. 看解码能否恢复原始逻辑比特。

这里我们保持参数：
- `n_bit = 3`  （每个相位块可以纠正最多 1 个 bit-flip）
- `n_phase = 3`（相位层可以纠正最多 1 个“相位标签错误”）

当然，当错误太多时（超出纠错能力），就会出现解码失败的情况。


In [20]:
import random

def demo_random_errors(num_logical_bits=3, n_bit=3, n_phase=3, num_errors=3, seed=None):
    if seed is not None:
        random.seed(seed)
    joint = BitPhaseJointCode(n_bit=n_bit, n_phase=n_phase)

    # 1) 随机生成逻辑比特
    logical = [random.randint(0, 1) for _ in range(num_logical_bits)]
    encoded = joint.encode(logical)

    # 2) 随机注入 num_errors 个错误
    received = encoded.copy()
    error_positions = random.sample(range(len(encoded)), num_errors)
    for pos in error_positions:
        received[pos] ^= 1

    # 3) 解码
    decoded, infos = joint.decode(received)

    print("逻辑比特：", logical)
    print("编码长度：", len(encoded))
    print("注入错误位置：", error_positions)
    print("解码结果：", decoded)
    print("是否完全纠正：", decoded == logical)

    return logical, encoded, received, decoded, infos


# 运行几次看看效果
for i in range(3):
    print(f"\n===== 实验 {i+1} =====")
    demo_random_errors(num_logical_bits=3, n_bit=3, n_phase=3, num_errors=3, seed=100 + i)



===== 实验 1 =====
逻辑比特： [0, 1, 1]
编码长度： 27
注入错误位置： [24, 5, 22]
解码结果： [[0], [1], [1]]
是否完全纠正： False

===== 实验 2 =====
逻辑比特： [0, 1, 1]
编码长度： 27
注入错误位置： [1, 21, 16]
解码结果： [[0], [1], [1]]
是否完全纠正： False

===== 实验 3 =====
逻辑比特： [0, 1, 0]
编码长度： 27
注入错误位置： [17, 22, 21]
解码结果： [[0], [1], [0]]
是否完全纠正： False


## 6. 小结

通过这个 Notebook，我们完成了：

1. **位重复码（BitRepetitionCode）**
   - 理解了通过多数表决纠正 bit-flip 的基本思想；
   - 实验了多比特序列在有错误时依然可以被恢复。

2. **相位重复码的经典模拟（PhaseRepetitionCode）**
   - 用 0/1 标签抽象“相位”，在经典框架中提前练习“相位纠错”的思路。

3. **位 + 相位联合纠错（BitPhaseJointCode）**
   - 构造了类似 Shor 3×3 结构的分层纠错方案：
     - 内层纠 bit-flip；
     - 外层纠 “phase 标签”；
   - 通过中间结果 `intermediate_phase_bits`，看清了两级纠错的分工。

下一步可以做的扩展包括：
- 引入真正的量子电路，构造 3-qubit bit-flip code 和 3-qubit phase-flip code；
- 实现一个真正的 Shor 9-qubit 量子码，并对照这个经典模型理解每一步操作。
